### Experiment #1: separating block of account from block of ip
#### At the moment, in case of access error we don't know whether account, ip or both were blocked. So we reach limit, and try to use them separately. This way we can find that one of the resources was not blocked and use it further.

In [1]:
import json


with open("../../resources/checkpoints/data_checkpoint.json") as f:
    user_data = json.load(f)

In [34]:
# will just retrieve groups walls
groups_to_get = user_data["142478661"]["groups"]

In [ ]:
proxy = input("Proxy address\n")
login = input("Login\n")
password = input("Password\n")

In [8]:
import sys
sys.path.insert(0, "../../")

In [21]:
from suvec.vk_api_impl.session.session_manager import SessionManager
from suvec.vk_api_impl.errors_handler import VkApiErrorsHandler
from suvec.vk_api_impl.session.records_managing import ProxyManager, CredsManager
from suvec.vk_api_impl.session.records_managing.terminal_out_of_records import TerminalOutOfProxy, TerminalOutOfCreds

import utils
from suvec.vk_api_impl.crawl_runner_with_checkpoints import VkCrawlRunnerWithCheckpoints
from suvec.common.events_tracking.terminal_events_tracker import TerminalEventsTracker
from suvec.vk_api_impl.session.records_managing.records_storing import ProxyStorage, CredsStorage
from suvec.vk_api_impl.session.records_managing.records_storing.serializers import ProxyRecordsSerializer, \
    CredsRecordsSerializer

In [23]:

settings_path = "../../settings.json"
proxies_save_pth, creds_save_pth = utils.get_proxy_and_creds_paths(settings_path)
checkp_data, checkp_requester = utils.get_data_requester_checkpoint_paths(settings_path)
result_file = utils.get_result_path(settings_path)
backups_path = utils.get_backups_path(settings_path)

proxy_storage = ProxyStorage(proxies_save_pth, ProxyRecordsSerializer())
creds_storage = CredsStorage(creds_save_pth, CredsRecordsSerializer())


In [26]:
errors_handler = VkApiErrorsHandler(None)

out_of_proxy_handler = TerminalOutOfProxy()
out_of_creds_handler = TerminalOutOfCreds()
proxy_manager = ProxyManager(proxy_storage, None, out_of_proxy_handler,
                             hours_for_resource_reload=24)
creds_manager = CredsManager(creds_storage, None, out_of_creds_handler,
                             hours_for_resource_reload=24)

session_manager = SessionManager(errors_handler, proxy_manager, creds_manager)

check record is usable <suvec.vk_api_impl.session.records_managing.records.ProxyRecord object at 0x7fbf5b952e20> 899.4991047382355
check record is usable <suvec.vk_api_impl.session.records_managing.records.ProxyRecord object at 0x7fbf5b952e80> 2133.8160860538483
check record is usable <suvec.vk_api_impl.session.records_managing.records.CredsRecord object at 0x7fbf5b9456a0> 899.5011949539185
check record is usable <suvec.vk_api_impl.session.records_managing.records.CredsRecord object at 0x7fbf5b945730> 2133.816159963608


In [41]:
session = session_manager._create_session(login, password, proxy)

In [ ]:
from time import sleep
from random import choice

cnt_requests = 0

def _make_request(group_id):
    request = session.vk_session.method("groups.getById", values={"group_ids": group_id})
    return request

while True:
    if cnt_requests % 30 == 0:
        print("cnt", cnt_requests)
    for _ in range(3):
        random_group_id = choice(groups_to_get)
        res = _make_request(random_group_id)
        cnt_requests += 1
    sleep(0)

cnt 0
cnt 30
cnt 60
cnt 90
cnt 120
cnt 150
cnt 180
cnt 210
cnt 240
cnt 270
cnt 300
cnt 330
cnt 360
cnt 390
cnt 420
cnt 450
cnt 480
cnt 510
cnt 540
cnt 570
cnt 600
cnt 630
cnt 660
cnt 690
cnt 720
cnt 750
cnt 780
cnt 810
cnt 840
cnt 870
cnt 900
cnt 930
cnt 960
cnt 990
cnt 1020
cnt 1050
cnt 1080
cnt 1110
cnt 1140
cnt 1170
cnt 1200
cnt 1230
cnt 1260
cnt 1290
cnt 1320
cnt 1350
cnt 1380
cnt 1410
cnt 1440
cnt 1470
cnt 1500
cnt 1530
cnt 1560
cnt 1590
cnt 1620
cnt 1650
cnt 1680
cnt 1710
cnt 1740
cnt 1770
cnt 1800
cnt 1830
cnt 1860
cnt 1890
cnt 1920
cnt 1950
cnt 1980
cnt 2010
cnt 2040
cnt 2070
cnt 2100
cnt 2130
cnt 2160
cnt 2190
cnt 2220
cnt 2250
cnt 2280
cnt 2310
cnt 2340
cnt 2370
cnt 2400
cnt 2430
cnt 2460
cnt 2490
cnt 2520
cnt 2550
cnt 2580
cnt 2610
cnt 2640
cnt 2670
cnt 2700
cnt 2730
cnt 2760
cnt 2790
cnt 2820
cnt 2850
cnt 2880
cnt 2910
cnt 2940
cnt 2970
cnt 3000
cnt 3030
cnt 3060
cnt 3090
cnt 3120
cnt 3150
cnt 3180
cnt 3210
cnt 3240
cnt 3270
cnt 3300
cnt 3330
cnt 3360
cnt 3390
cnt 3420
cnt 

In [54]:
print(res)

[{'id': 188533997, 'name': 'Проектный практикум', 'screen_name': 'project__it', 'is_closed': 0, 'type': 'page', 'is_admin': 0, 'is_member': 0, 'is_advertiser': 0, 'photo_50': 'https://sun6-23.userapi.com/s/v1/ig2/JGlmuH32QtuqdnIfNRYmORz6H3Hnu1Zrsb830pi1YhBKC7NI8Sxo7ZlvPh-GanNoQqjDIfzU14JW9_rFsw4VWDon.jpg?size=50x0&quality=96&crop=59,51,717,717&ava=1', 'photo_100': 'https://sun6-23.userapi.com/s/v1/ig2/FFPtIAzDmpQbkPg3HYkdZwNlLKyHkW6935Xq6yuLuELGK59bcD3X1s7vaIuU_fY0scM6E4d1BQVeMy0dBZx66qw2.jpg?size=100x0&quality=96&crop=59,51,717,717&ava=1', 'photo_200': 'https://sun6-23.userapi.com/s/v1/ig2/4ZA_SS-IhMXSEVKpLSXVcDPLQMpnJhnmnHvng_paUSwDaOpcycY6eyZNoo5tvNmrGs-9sK4MC87L87kV9PRNvEGr.jpg?size=200x0&quality=96&crop=59,51,717,717&ava=1'}]
